# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Monday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**

**Please check that the link you submitted can be opened and points to the correct assignment.**


# Question 1 (25 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]


(3) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(4) Collect all the information of the 904 narrators in the Densho Digital Repository.

(5)**Collect a total of 10000 reviews** of the top 100 most popular software from G2 and Capterra.


In [1]:
!pip install selenium
!pip install webdriver-manager

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 8.8 MB/s eta 0:00:00


##Reference - https://stackoverflow.com/questions/51046454/how-can-we-use-selenium-webdriver-in-colab-research-google-com##

In [2]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.3kSz9SES1h/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.gojCs2YRY8/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.f6xy2AxWSd/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://security.ubuntu.com/ubuntu

In [5]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#from webdriver_manager.chrome import ChromeDriverManager # We don't need this if using apt for driver
import time
import pandas as pd

# Setting up Chrome browser options for headless mode (no GUI)
browser_settings = Options()
browser_settings.add_argument('--headless')  # Run without opening a browser window
browser_settings.add_argument('--no-sandbox')  # Required for some environments
browser_settings.add_argument('--disable-dev-shm-usage')  # Prevent memory issues
browser_settings.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36")
browser_settings.add_argument('--blink-settings=imagesEnabled=false')  # Turn off images for faster loading


# Initialize the WebDriver (this is what controls the browser)
# For Colab environments, often /usr/bin/chromedriver is the location. Adjust if needed.
driver_path = "/usr/bin/chromedriver"
# If driver_path does NOT work, you may need to find the actual location in your system:
# !which chromedriver
driver_service = Service(driver_path)
driver = webdriver.Chrome(service=driver_service, options=browser_settings)

Hit:1 http://deb.debian.org/debian buster InRelease
Hit:2 http://deb.debian.org/debian buster-updates InRelease
Hit:3 http://deb.debian.org/debian-security buster/updates InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 

In [6]:
# movie  - THE Dark Knight Rises
movie_reviews_url = "https://www.imdb.com/title/tt1345836/reviews/"
driver.get(movie_reviews_url)

# Scrolling to the bottom of the page to ensure all elements are loaded
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(2)  # Wait for 2 seconds to allow elements like buttons to load

try:
    show_all_button = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, "//button//span[text()='All']"))
    )
    driver.execute_script("arguments[0].click();", show_all_button)
    time.sleep(5)  # Wait for reviews to load after clicking "All"
except Exception as e:
    print(f"Couldn't click 'All' button due to: {e}")


In [7]:

# Initialize an empty list to store reviews and set scraping parameters
reviews_list = []
desired_review_count = 1000  # Target number of reviews to scrape
batch_review_count = 25  # Reviews loaded per batch when clicking "25 more"
wait_time_between_batches = 5  # Wait time between loading more reviews
max_retries = 3  # Maximum retries if "25 more" button fails

In [8]:
# Start scraping reviews in batches until desired count is reached
while len(reviews_list) < desired_review_count:
    try:
        # Find all review elements currently visible on the page
        review_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, "//div[@class='ipc-html-content-inner-div']"))
        )
        for review_element in review_elements:
            if len(reviews_list) >= desired_review_count:
                break  # Stop collecting if we've reached our target count

            review_text = review_element.text.strip()  # Extract review text and clean it up
            reviews_list.append(review_text)

        # If we still need more reviews, try loading additional ones by clicking "25 more"
        if len(reviews_list) < desired_review_count:
            retry_attempts = 0
            while retry_attempts < max_retries:
                try:
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")  # Scroll down
                    time.sleep(2)  # Wait for button visibility

                    load_more_button = WebDriverWait(driver, 5).until(
                        EC.element_to_be_clickable((By.XPATH, "//button//span[contains(text(), '25 more')]"))
                    )
                    driver.execute_script("arguments[0].click();", load_more_button)
                    time.sleep(wait_time_between_batches)  # Wait for new reviews to load
                    break  # Exit retry loop if successful
                except Exception as retry_error:
                    retry_attempts += 1
                    time.sleep(5)  # Wait before retrying again

            else:  # If retries fail completely, stop scraping further
                break

    except Exception as scrape_error:
        print(f"Error during scraping: {scrape_error}")
        break

# Close the browser session after scraping is complete
driver.quit()

In [9]:
# Save scraped reviews into a CSV file using pandas DataFrame
reviews_df = pd.DataFrame(reviews_list, columns=["review"])
reviews_df.to_csv('imdb_scraped_reviews.csv', index=False)

print(f"Scraping completed! Total reviews collected: {len(reviews_list)}. Data saved in 'imdb_scraped_reviews.csv'.")

Scraping completed! Total reviews collected: 1000. Data saved in 'imdb_scraped_reviews.csv'.


In [10]:
reviews_df

,review
0,"After eight years in seclusion, Batman resurfa..."
1,The Dark Knight Rises was a perfect sequel to ...
2,Possibly my personal favourite of the Nolan Ba...
3,There's a storm coming .....\n\nYears after be...
4,Christopher Nolan's epic trilogy concludes in ...
...,...
995,It's been eight years since the events of the ...
996,Got to watch it in a preview show here in Indi...
997,Another achievement by Nolan Bros. and Warner ...
998,"This movie is full of excitement, explosions, ..."


# Question 2 (15 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [11]:
# Write code for each of the sub parts with proper comments.

import re, string, nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Download necessary NLTK data (do this only once)
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("omw-1.4")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [12]:
# Define a cleaning pipeline function that does each step individually
def my_text_cleaner(raw_text):
    work_steps = {}  # dictionary to store intermediate results

    # Step 0: Save the original text.
    work_steps["original"] = raw_text

    # Step 1: Remove special characters and punctuations.
    # We remove any punctuation defined in string.punctuation.
    no_punc = re.sub(r"[" + re.escape(string.punctuation) + r"]", " ", raw_text)
    work_steps["no_punctuation"] = no_punc

    # Step 2: Remove numbers using a regex.
    no_nums = re.sub(r"\d+", "", no_punc)
    work_steps["no_numbers"] = no_nums

    # Step 3: Remove stopwords.
    word_list = no_nums.split()  # split on whitespace
    stop_set = set(stopwords.words("english"))
    # We check lower-case for stopword removal to catch all cases.
    no_stops = [w for w in word_list if w.lower() not in stop_set]
    work_steps["no_stopwords"] = " ".join(no_stops)

    # Step 4: Lowercase all texts.
    lower_text = work_steps["no_stopwords"].lower()
    work_steps["lowercase"] = lower_text

    # Step 5: Apply stemming using PorterStemmer.
    port_stemmer = PorterStemmer()
    stemed_words = [port_stemmer.stem(w) for w in lower_text.split()]
    stemed_text = " ".join(stemed_words)
    work_steps["stemming"] = stemed_text

    # Step 6: Apply lemmatization using WordNetLemmatizer.
    wordnet_lem = WordNetLemmatizer()
    lem_words = [wordnet_lem.lemmatize(w) for w in stemed_text.split()]
    lem_text = " ".join(lem_words)
    work_steps["lemmatization"] = lem_text

    return work_steps

In [13]:
raw_df = pd.read_csv("imdb_scraped_reviews.csv")

# Step by Step changes for the reviews
if not raw_df.empty:
    first_rev = raw_df["review"].iloc[0]
    print("Original review text:")
    print(first_rev)
    print("\n--- Cleaning steps for the first review ---")
    steps_out = my_text_cleaner(first_rev)
    for step, out_txt in steps_out.items():
        print(f"\n[{step}]:")
        print(out_txt)

Original review text:
After eight years in seclusion, Batman resurfaces to face Bane, a mastermind bent on destroying Gotham and has ties to Bruce Wayne's past. Christopher Nolan's conclusion to his Dark Knight trilogy should be treated as a wrap up rather than a third installment. Despite many new characters being introduced, even their conflicts and motives are derived from events that occurred in the previous two films. Such an approach could be seen as a weakness, since the conflict could just end up being sort of a rematch rather than moving the story forward; however, Nolan's approach works terrificly. Over an hour is shot in stunning IMAX, but the film is more touching than it is intense and that's its best quality. Hardy steps in as the genius mercenary Bane, who is slightly bigger, slightly stronger and slightly smarter than Batman, thus making him a challenge best suited for the grand finale. The Joker works best as Batman's arch-nemesis since they are polar opposites, with 

In [14]:
# Now, apply the cleaning pipeline to all reviews and store the final cleaned text
# We are taking only the 'lemmatization' output as the final cleaned version.
raw_df["clean_review"] = raw_df["review"].apply(lambda txt: my_text_cleaner(txt)["lemmatization"])

In [15]:
raw_df.to_csv("imdb_scraped_reviews_clean.csv", index=False)
raw_df

,review,clean_review
0,"After eight years in seclusion, Batman resurfa...",eight year seclus batman resurfac face bane ma...
1,The Dark Knight Rises was a perfect sequel to ...,dark knight rise perfect sequel masterpiec chr...
2,Possibly my personal favourite of the Nolan Ba...,possibl person favourit nolan batman film like...
3,There's a storm coming .....\n\nYears after be...,storm come year villifi batman encourag come g...
4,Christopher Nolan's epic trilogy concludes in ...,christoph nolan epic trilog conclud gloriou fa...
...,...,...
995,It's been eight years since the events of the ...,eight year sinc event immedi predecessor film ...
996,Got to watch it in a preview show here in Indi...,got watch preview show india stun way nolan na...
997,Another achievement by Nolan Bros. and Warner ...,anoth achiev nolan bro warner bro thank huge b...
998,"This movie is full of excitement, explosions, ...",movi full excit explos edg seat moment tie loo...


# Question 3 (15 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [16]:
!pip install spacy benepar pandas
!python -m spacy download en_core_web_sm
!python -m benepar.download benepar_en3

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.4 MB/s eta 0:00:00
  Created wheel for benepar: filename=benepar-0.2.0-py3-none-any.whl size=37626 sha256=d492ac7998a993a44e8a5c96404a96f8fab12c1f8bada9b8a9af20e451cd8816
  Stored 

In [17]:
# Your code here
import spacy
import benepar
import pandas as pd
from collections import Counter


In [18]:
# Load SpaCy model
benepar.download('benepar_en3')

nlp_big = spacy.load("en_core_web_sm")
if "benepar" not in nlp_big.pipe_names:
    nlp_big.add_pipe("benepar", config={"model": "benepar_en3"})

[nltk_data] Downloading package benepar_en3 to /root/nltk_data...
[nltk_data]   Unzipping models/benepar_en3.zip.
/usr/local/lib/python3.11/dist-packages/benepar/parse_chart.py:169: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub 

In [19]:
df_reviews = pd.read_csv("imdb_scraped_reviews_clean.csv")

In [20]:
df_reviews

,review,clean_review
0,"After eight years in seclusion, Batman resurfa...",eight year seclus batman resurfac face bane ma...
1,The Dark Knight Rises was a perfect sequel to ...,dark knight rise perfect sequel masterpiec chr...
2,Possibly my personal favourite of the Nolan Ba...,possibl person favourit nolan batman film like...
3,There's a storm coming .....\n\nYears after be...,storm come year villifi batman encourag come g...
4,Christopher Nolan's epic trilogy concludes in ...,christoph nolan epic trilog conclud gloriou fa...
...,...,...
995,It's been eight years since the events of the ...,eight year sinc event immedi predecessor film ...
996,Got to watch it in a preview show here in Indi...,got watch preview show india stun way nolan na...
997,Another achievement by Nolan Bros. and Warner ...,anoth achiev nolan bro warner bro thank huge b...
998,"This movie is full of excitement, explosions, ...",movi full excit explos edg seat moment tie loo...


POS Tagging

In [21]:
all_sents_collected = []

#process a text, and if error occurs, it will split the text manually.
def safe_sentence_extraction(raw_txt):
    sent_list = []
    word_limit = 300  # splitting threshold (approximately)
    # If the review is huge, break it apart.
    words = raw_txt.split()
    if len(words) > word_limit:
        # Process in chunks (each chunk is at most word_limit words)
        for i in range(0, len(words), word_limit):
            chunk = " ".join(words[i:i+word_limit])
            try:
                doc_chunk = nlp_big(chunk)
                # Use SpaCy's sentence segmentation on the chunk
                sents = list(doc_chunk.sents)
                sent_list.extend(sents)
            except Exception as err_chunk:
                print("Skipped a chunk due to error:", err_chunk)
    else:
        try:
            doc_full = nlp_big(raw_txt)
            sent_list.extend(list(doc_full.sents))
        except Exception as err_full:
            # Fallback: split manually on period if processing fails
            print("Error encountered when processing a review. Splitting manually.")
            parts = raw_txt.split('.')
            for part in parts:
                part = part.strip()
                if part:
                    try:
                        doc_part = nlp_big(part)
                        sent_list.extend(list(doc_part.sents))
                    except Exception as err_part:
                        print("Still got an error on a smaller piece; skipping that piece.")
    return sent_list

In [22]:
reviews_df = pd.read_csv("imdb_scraped_reviews_clean.csv")

# Loop over each review and extract its sentences safely
all_sentences = []
for one_review in reviews_df["clean_review"]:
    sents_from_review = safe_sentence_extraction(one_review)
    all_sentences.extend(sents_from_review)

print(f"Total valid sentences collected: {len(all_sentences)}")

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.11/dist-packages/torch/distributions/distribution.py:56: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (526 > 512). Running this sequence through the model will result in indexing errors


Skipped a chunk due to error: Sentence of length 527 (in sub-word tokens) exceeds the maximum supported length of 512
Skipped a chunk due to error: Sentence of length 527 (in sub-word tokens) exceeds the maximum supported length of 512
Skipped a chunk due to error: Sentence of length 537 (in sub-word tokens) exceeds the maximum supported length of 512
Skipped a chunk due to error: Sentence of length 527 (in sub-word tokens) exceeds the maximum supported length of 512
Skipped a chunk due to error: Sentence of length 537 (in sub-word tokens) exceeds the maximum supported length of 512
Skipped a chunk due to error: Sentence of length 518 (in sub-word tokens) exceeds the maximum supported length of 512
Error encountered when processing a review. Splitting manually.
Still got an error on a smaller piece; skipping that piece.
Skipped a chunk due to error: Sentence of length 523 (in sub-word tokens) exceeds the maximum supported length of 512
Skipped a chunk due to error: Sentence of length 5

In [23]:
all_sentences[0]

eight year seclus batman resurfac face bane mastermind bent destroy gotham tie bruce wayn past christoph nolan conclus dark knight trilog treat wrap rather third instal despit mani new charact introduc even conflict motiv deriv event occur previou two film approach could seen weak sinc conflict could end sort rematch rather move stori forward howev nolan approach work terrificli hour shot stun imax film touch intens best qualiti hardi step geniu mercenari bane slightli bigger slightli stronger slightli smarter batman thu make challeng best suit grand final joker work best batman arch nemesi sinc polar opposit joker stand everyth batman stand two face work best batman tragic enemi harvey dent much like bruce wayn differ dent cope tragedi reveng rather seek justic wayn bane work best batman challeng villain physic mental superiority—forget bane saw batman robin nolan care chose villain seri wise execut perfectli hardi big shoe fill follow ledger posthum oscar win perform bane joker hardi

In [24]:
# Instead of combining all sentences, process them individually:
noun_count = 0
verb_count = 0
adj_count = 0
adv_count = 0

for sent in all_sentences:
    # Check if the sentence is too long for Benepar
    if len(sent) > 512:  # Assuming 512 is the token limit for Benepar
        print(f"Skipping sentence due to length: {sent.text[:50]}...")
        continue  # Skip this sentence

    for tok in sent:
        if tok.pos_ == "NOUN":
            noun_count += 1
        elif tok.pos_ == "VERB":
            verb_count += 1
        elif tok.pos_ == "ADJ":
            adj_count += 1
        elif tok.pos_ == "ADV":
            adv_count += 1

print("\n=== POS Tagging Counts ===")
print("Total Nouns:", noun_count)
print("Total Verbs:", verb_count)
print("Total Adjectives:", adj_count)
print("Total Adverbs:", adv_count)


=== POS Tagging Counts ===
Total Nouns: 50494
Total Verbs: 21141
Total Adjectives: 19482
Total Adverbs: 5752


In [25]:
# Constituency and Dependency Parsing:
# We choose one example sentence (first valid sentence) for detailed parsing.

if len(all_sentences) > 0:
    sample_sent = all_sentences[0]
    print("\n=== Sample Sentence for Parsing ===")
    print(sample_sent.text)

    # Constituency Parsing via benepar: print the parse tree string.
    if hasattr(sample_sent._, "parse_string"):
        print("\n--- Constituency Parse Tree ---")
        print(sample_sent._.parse_string)
    else:
        print("No constituency parse available for this sentence.")

    # Dependency Parsing: print dependencies for each token.
    print("\n--- Dependency Parsing Details ---")
    for token in sample_sent:
        print(f"Word: {token.text}, POS: {token.pos_}, Dependency: {token.dep_}, Head: {token.head.text}")

    # Explanation:
    print("\nExplanation:")
    print("The Constituency Parse Tree shows the hierarchical phrase structure of the sentence. It breaks the sentence")
    print("into nested constituents like noun phrases (NP) and verb phrases (VP), reflecting the grammatical structure.")
    print("On the other hand, the Dependency Parse output lists how each word is related directly to another (its 'head')")
    print("with dependency labels, revealing the syntactic relationships between words.")
else:
    print("No sentences available for parsing.")


=== Sample Sentence for Parsing ===
eight year seclus batman resurfac face bane mastermind bent destroy gotham tie bruce wayn past christoph nolan conclus dark knight trilog treat wrap rather third instal despit mani new charact introduc even conflict motiv deriv event occur previou two film approach could seen weak sinc conflict could end sort rematch rather move stori forward howev nolan approach work terrificli hour shot stun imax film touch intens best qualiti hardi step geniu mercenari bane slightli bigger slightli stronger slightli smarter batman thu make challeng best suit grand final joker work best batman arch nemesi sinc polar opposit joker stand everyth batman stand two face work best batman tragic enemi harvey dent much like bruce wayn differ dent cope tragedi reveng rather seek justic wayn bane work best batman challeng villain physic mental superiority—forget bane saw batman robin nolan care chose villain seri wise execut perfectli hardi big shoe fill follow ledger posth

In [26]:
# Named Entity Recognition (NER):
entity_counter = Counter()  # Initialize an empty counter

for sent in all_sentences:
    # Check if the sentence is too long for Benepar
    if len(sent) > 512:  # 512 is the token limit for Benepar
        print(f"Skipping sentence due to length for NER: {sent.text[:50]}...")
        continue  # Skip this sentence for NER

    doc_ner = nlp_big(sent.text)  # Process the sentence with SpaCy
    entity_counter.update([ent.label_ for ent in doc_ner.ents])  # Update counter

print("\n=== Named Entity Recognition (NER) Counts ===")
for entity_label, count in entity_counter.items():
    print(f"{entity_label}: {count}")


=== Named Entity Recognition (NER) Counts ===
DATE: 496
PERSON: 5456
ORG: 1484
ORDINAL: 771
CARDINAL: 1656
FAC: 37
NORP: 1070
GPE: 362
TIME: 205
LOC: 10
WORK_OF_ART: 25
PRODUCT: 84
QUANTITY: 9
LAW: 4
MONEY: 5
EVENT: 1


# **Following Questions must answer using AI assitance**

#Question 4 (20 points).

Q4. (PART-1)
Web scraping data from the GitHub Marketplace to gather details about popular actions. Using Python, the process begins by sending HTTP requests to multiple pages of the marketplace (1000 products), handling pagination through dynamic page numbers. The key details extracted include the product name, a short description, and the URL.

 The extracted data is stored in a structured CSV format with columns for product name, description, URL, and page number. A time delay is introduced between requests to avoid server overload. ChatGPT can assist by helping with the parsing of HTML, error handling, and generating reports based on the data collected.

 The goal is to complete the scraping within a specified time limit, ensuring that the process is efficient and adheres to GitHub’s usage guidelines.

(PART -2)

1.   **Preprocess Data**: Clean the text by tokenizing, removing stopwords, and converting to lowercase.

2. Perform **Data Quality** operations.


Preprocessing:
Preprocessing involves cleaning the text by removing noise such as special characters, HTML tags, and unnecessary whitespace. It also includes tasks like tokenization, stopword removal, and lemmatization to standardize the text for analysis.

Data Quality:
Data quality checks ensure completeness, consistency, and accuracy by verifying that all required columns are filled and formatted correctly. Additionally, it involves identifying and removing duplicates, handling missing values, and ensuring the data reflects the true content accurately.


Github MarketPlace page:
https://github.com/marketplace?type=actions

In [28]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup

nltk.download('punkt')
nltk.download('stopwords')

chrome_options = Options()
chrome_options.add_argument('--headless')         # Run without opening the browser window.
chrome_options.add_argument('--no-sandbox')         # Needed in some environments.
chrome_options.add_argument('--disable-dev-shm-usage')# Prevent shared memory issues.
# (Additional options, if needed, could be added here.)

# Install specific chromedriver
!apt-get install chromium-chromedriver

# Set the path to the downloaded ChromeDriver
driver_path = "/usr/bin/chromedriver"  # or where you extracted the downloaded driver
driver_service = Service(driver_path)
wd = webdriver.Chrome(service=driver_service, options=chrome_options)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  chromium-chromedriver
0 upgraded, 1 newly installed, 0 to remove and 22 not upgraded.
1 not fully installed or removed.
Need to get 2,308 B of archives.
After this operation, 77.8 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 chromium-chromedriver amd64 1:85.0.4183.83-0ubuntu2.22.04.1 [2,308 B]
Fetched 2,308 B in 0s (37.5 kB/s)
(Reading database ... 125730 files and directories currently installed.)
Preparing to unpack .../chromium-chromedriver_1%3a85.0.4183.83-0ubuntu2.22.04.1_amd64.deb ...
Unpacking chromium-chromedriver (1:85.0.4183.83-0ubuntu2.22.04.1) ...
dpkg: error processing archive /var/cache/apt/archives/chromium-chromedriver_1%3a85.0.4183.83-0ubuntu2.22.04.1_amd64.deb (--unpack):
 trying to overwrite '/usr/bin/chromedriver', which is also in package chromium-driver 90.0

In [29]:
product_data = []

# Loop through pages in the GitHub Marketplace. In this example, we iterate through 50 pages.

for page in range(1, 51):
    # Build the URL with pagination and filtering for GitHub Actions.
    page_url = f"https://github.com/marketplace?page={page}&type=actions"
    print(f"Processing page {page}: {page_url}")

    # Use Selenium to load the page.
    wd.get(page_url)
    time.sleep(3)  # Pause to let the page complete loading.

    # Wait for the product links to appear (checking for links that include '/marketplace/actions/').
    try:
        product_link_elements = WebDriverWait(wd, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a[href*='/marketplace/actions/']"))
        )
    except Exception as e:
        print(f"Error waiting for product links on page {page}: {e}")
        continue  # Go to the next page if products are not found.

    # Extract the href attribute from each link element.
    product_hrefs = [elem.get_attribute("href") for elem in product_link_elements]


    # For each product URL, navigate and extract product details.

    for prod_href in product_hrefs:
        # Visit the product page.
        wd.get(prod_href)
        time.sleep(3)  # Allow time for the page to load.

        try:
            # Derive product name from the URL.
            # Example: if URL ends with "cool-action", convert it to "cool action".
            prod_name = prod_href.split("/")[-1].replace("-", " ")

            # Wait for the product’s description to load.
            description_el = WebDriverWait(wd, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".markdown-body"))
            )

            # Extract and clean the full text from the description element.
            full_text = description_el.text.strip()
            description_clean = " ".join(full_text.split('\n'))  # remove extra newlines

            # Append the details (name, URL, description) and current page to our list.
            product_data.append([prod_name, prod_href, description_clean, page])
        except Exception as error_message:
            print(f"Error scraping description for {prod_href}: {error_message}")
            # If error, use placeholder values.
            prod_name = prod_href.split("/")[-1].replace("-", " ")
            product_data.append([prod_name, prod_href, "Description not found", page])

    print(f"Total products scraped so far: {len(product_data)}")
    time.sleep(2)  # Polite delay before going to the next page.

    # If we've reached our target (or near 1000), you can break out of the loop.
    if len(product_data) >= 1000:
        break


# Cleanup - close the browser session.

wd.quit()

df_products = pd.DataFrame(product_data, columns=["Product Name", "Product URL", "Description", "Page Number"])
csv_filename = "github_marketplace_products.csv"
df_products.to_csv(csv_filename, index=False)

# Display the DataFrame (or its head) for verification.
print("\nDataframe with the extracted product information:")
print(df_products.head())
print(f"\nFinished scraping. Total {len(product_data)} products saved to '{csv_filename}'.")

Processing page 1: https://github.com/marketplace?page=1&type=actions
Total products scraped so far: 20
Processing page 2: https://github.com/marketplace?page=2&type=actions
Total products scraped so far: 40
Processing page 3: https://github.com/marketplace?page=3&type=actions
Total products scraped so far: 60
Processing page 4: https://github.com/marketplace?page=4&type=actions
Total products scraped so far: 80
Processing page 5: https://github.com/marketplace?page=5&type=actions
Total products scraped so far: 100
Processing page 6: https://github.com/marketplace?page=6&type=actions
Total products scraped so far: 120
Processing page 7: https://github.com/marketplace?page=7&type=actions
Total products scraped so far: 140
Processing page 8: https://github.com/marketplace?page=8&type=actions
Total products scraped so far: 160
Processing page 9: https://github.com/marketplace?page=9&type=actions
Total products scraped so far: 180
Processing page 10: https://github.com/marketplace?page=10&

In [30]:
df_products

,Product Name,Product URL,Description,Page Number
0,trufflehog oss,https://github.com/marketplace/actions/truffle...,TruffleHog Find leaked credentials. 🔎 Now Scan...,1
1,metrics embed,https://github.com/marketplace/actions/metrics...,📊 Metrics Generate metrics that can be embedde...,1
2,yq portable yaml processor,https://github.com/marketplace/actions/yq-port...,yq a lightweight and portable command-line YAM...,1
3,super linter,https://github.com/marketplace/actions/super-l...,Super-Linter Super-linter is a ready-to-run co...,1
4,gosec security checker,https://github.com/marketplace/actions/gosec-s...,gosec - Go Security Checker Inspects source co...,1
...,...,...,...,...
995,notify microsoft teams,https://github.com/marketplace/actions/notify-...,Notify Microsoft Teams Teams notify action ins...,50
996,setup matlab,https://github.com/marketplace/actions/setup-m...,Action for Setting Up MATLAB The Setup MATLAB ...,50
997,rworkflows,https://github.com/marketplace/actions/rworkflows,"Authors: Brian Schilder, Alan Murphy, Nathan S...",50
998,generate sitemap,https://github.com/marketplace/actions/generat...,generate-sitemap Check out all of our GitHub A...,50


In [31]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


In [32]:
nltk.download("punkt_tab")
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [33]:
# Define a function that cleans a string using the steps outlined above.
def preprocess_text(raw_text):
    # Step 1: Remove HTML tags (if there are any) using regex
    text_no_html = re.sub(r'<.*?>', ' ', raw_text)

    # Step 2: Remove special characters and punctuation.
    # This keeps only letters, numbers, and whitespace.
    text_no_special = re.sub(r'[^a-zA-Z0-9\s]', '', text_no_html)

    # Step 3: Normalize whitespace - convert multiple spaces/newlines to a single space.
    text_clean = re.sub(r'\s+', ' ', text_no_special)

    # Step 4: Convert the text to lowercase.
    text_lower = text_clean.lower()

    # Step 5: Tokenize the text into words using NLTK’s word_tokenize.
    tokens = word_tokenize(text_lower)

    # Step 6: Remove stopwords using the English stopwords list.
    stopw = set(stopwords.words("english"))
    tokens_no_stop = [word for word in tokens if word not in stopw]

    # Step 7: Lemmatize tokens to get their dictionary form using WordNetLemmatizer.
    lemmatizer = WordNetLemmatizer()
    tokens_lemmatized = [lemmatizer.lemmatize(token) for token in tokens_no_stop]

    # Finally, join the tokens back together into one cleaned text string.
    final_text = " ".join(tokens_lemmatized)

    return final_text

In [34]:
df = pd.read_csv("github_marketplace_products.csv")

# Create a new column "clean_description" by applying our preprocess_text() function
df["clean_description"] = df["Description"].apply(lambda txt: preprocess_text(txt) if isinstance(txt, str) else "")

# Data Quality check: Ensure essential columns are not missing.
required_cols = ["Product Name", "Product URL", "Description"]
df_quality = df.dropna(subset=required_cols)

# Optional: Trim extra spaces from string columns for consistency.
for col in required_cols:
    df_quality[col] = df_quality[col].astype(str).str.strip()

# Remove duplicates if any exist.
df_quality = df_quality.drop_duplicates()

# Save the cleaned and quality-checked data to a new CSV file.
output_filename = "github_marketplace_products_clean.csv"
df_quality.to_csv(output_filename, index=False)

print("Preprocessing and data quality operations complete.")
print(f"Clean data saved to '{output_filename}'.")

Preprocessing and data quality operations complete.
Clean data saved to 'github_marketplace_products_clean.csv'.


#Question 5 (20 points)

PART 1:
Web Scrape  tweets from Twitter using the Tweepy API, specifically targeting hashtags related to subtopics (machine learning or artificial intelligence.)
The extracted data includes the tweet ID, username, and text.

Part 2:
Perform data cleaning procedures

A final data quality check ensures the completeness and consistency of the dataset. The cleaned data is then saved into a CSV file for further analysis.


**Note**

1.   Follow tutorials provided in canvas to obtain api keys. Use ChatGPT to get the code. Make sure the file is downloaded and saved.
2.   Make sure you divide GPT code as shown in tutorials, dont make multiple requestes.


In [35]:
import tweepy
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download required NLTK resources (if not already downloaded)
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [36]:
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAANNNzQEAAAAAyUEEfuKkZJpn8ZzLpejK0Eky%2Fq0%3DtQBcIFnGd4AINalPuWyFOYmdWHOjFC5WzqnzPwyIaZSwnmSKZc"
CONSUMER_KEY = "qZXfpdDhQsJp2qbYiF4YRxsfu"
CONSUMER_SECRET = "84pRH4dD78YroSUQs4BSG6YCSDplB5NdDVpEfl4X7lpMtZxuFx"
ACCESS_TOKEN = "1891238056499286017-d1VXyBeBP5eABMc9ZKtXGtCYtAPY2F"
ACCESS_TOKEN_SECRET = "KUs4idmTvB6TUwXvHGhfFK4vCrfZD2MtdCoW5yUnCqT9X"



In [42]:
# Initialize the Tweepy client with your credentials.
client = tweepy.Client(
    bearer_token=BEARER_TOKEN,
    consumer_key=CONSUMER_KEY,
    consumer_secret=CONSUMER_SECRET,
    access_token=ACCESS_TOKEN,
    access_token_secret=ACCESS_TOKEN_SECRET,
    # Handling Rate Limits: These options tell Tweepy to respect Twitter's rate limits
    wait_on_rate_limit=True,
    # wait_on_rate_limit_notify=True
)

In [43]:
# We want tweets containing either #machinelearning or #artificialintelligence.
# Additionally, we exclude retweets and restrict to English-language tweets.
query = "#machinelearning OR #artificialintelligence -is:retweet lang:en"

# We need specific tweet and user fields.
tweet_fields = ["id", "text", "author_id"]
user_fields = ["username"]
expansions = "author_id"  # This returns user objects in the response


In [ ]:
tweets_list = []
users_dict = {}

# Implementing a loop to make multiple requests and handle rate limits
max_tweets = 100  # Set the desired number of tweets
tweets_fetched = 0

while tweets_fetched < max_tweets:
    try:
        response = client.search_recent_tweets(
            query=query,
            tweet_fields=tweet_fields,
            user_fields=user_fields,
            expansions=expansions,
            max_results=10  # Maximum tweets per request
        )
        # Process the response, extract tweets, and update counters
        if response.data:
            tweets_list.extend(response.data)
            tweets_fetched += len(response.data)
        if response.includes and "users" in response.includes:
            for user in response.includes["users"]:
                users_dict[user.id] = user.username
        print(f"Fetched {tweets_fetched} tweets so far...")
        # Introduce a delay to avoid hitting rate limits too quickly
        time.sleep(20)  # Adjust as needed
    except tweepy.TooManyRequests as e:
        print("Rate limit hit. Waiting for 15 minutes...")
        time.sleep(900)  # Wait for 15 minutes
    except Exception as e:
        print(f"Error occurred: {e}")
        break  # Exit the loop if an unexpected error occurs

print("Number of tweets scraped:", len(tweets_list))

Fetched 10 tweets so far...


Fetched 20 tweets so far...


Fetched 30 tweets so far...


Fetched 40 tweets so far...


Fetched 50 tweets so far...


Fetched 60 tweets so far...


Fetched 70 tweets so far...


Fetched 80 tweets so far...


In [57]:
# Build DataFrame
df = pd.DataFrame([{
    "tweet_id": tweet.id,
    "username": users_dict.get(tweet.author_id, "Unknown"),
    "text": tweet.text,
    "created_at": tweet.created_at
} for tweet in tweets_list])

Save results
df.to_csv("tweets.csv", index=False)
print(f"Saved {len(df)} tweets to tweets.csv")

In [58]:
df

,tweet_id,username,text,created_at
0,1891299408085401715,Paul_hack13,You need help with your account recovery kindl...,None
1,1891299346622148795,Property_Report,Updates: Phuket luxury market lures foreign in...,None
2,1891299238291611988,notredamuz9,"@MrZackMorris ETF500's price may be volatile, ...",None
3,1891297982756102281,Webcrack01,Is mandatory password expiration helping or hu...,None
4,1891297817269788738,Recoverytheate,Is mandatory password expiration helping or hu...,None
...,...,...,...,...
95,1891332177725456767,tut_ml,12 Best Courses to Learn Deep Learning!\nhttps...,None
96,1891331968819773592,Theo_Zou,RT @RemoteSens_MDPI: #mostcited\nEstimation of...,None
97,1891331897785008232,tut_ml,FREE Statistics Courses- https://t.co/04Hg1nfO...,None
98,1891331745250730163,kdagiit,"Data prep is like cooking, model training is l...",None


In [59]:
def clean_tweet(text):
    # Remove URLs
    text = re.sub(r'http\S+|www.\S+', '', text)

    # Remove mentions (@username)
    text = re.sub(r'@\w+', '', text)

    # Remove hashtags (#) but keep the text
    text = re.sub(r'#', '', text)

    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Convert to lowercase
    text = text.lower()

    # Tokenize
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Join tokens back into text
    clean_text = ' '.join(tokens)

    return clean_text

In [60]:
#Remove any duplicate tweets based on tweet_id
df = df.drop_duplicates(subset=['tweet_id'])
df

,tweet_id,username,text,created_at
0,1891299408085401715,Paul_hack13,You need help with your account recovery kindl...,None
1,1891299346622148795,Property_Report,Updates: Phuket luxury market lures foreign in...,None
2,1891299238291611988,notredamuz9,"@MrZackMorris ETF500's price may be volatile, ...",None
3,1891297982756102281,Webcrack01,Is mandatory password expiration helping or hu...,None
4,1891297817269788738,Recoverytheate,Is mandatory password expiration helping or hu...,None
...,...,...,...,...
95,1891332177725456767,tut_ml,12 Best Courses to Learn Deep Learning!\nhttps...,None
96,1891331968819773592,Theo_Zou,RT @RemoteSens_MDPI: #mostcited\nEstimation of...,None
97,1891331897785008232,tut_ml,FREE Statistics Courses- https://t.co/04Hg1nfO...,None
98,1891331745250730163,kdagiit,"Data prep is like cooking, model training is l...",None


In [62]:
#Remove rows with missing values
df = df.drop(columns=['created_at'])
df


,tweet_id,username,text
0,1891299408085401715,Paul_hack13,You need help with your account recovery kindl...
1,1891299346622148795,Property_Report,Updates: Phuket luxury market lures foreign in...
2,1891299238291611988,notredamuz9,"@MrZackMorris ETF500's price may be volatile, ..."
3,1891297982756102281,Webcrack01,Is mandatory password expiration helping or hu...
4,1891297817269788738,Recoverytheate,Is mandatory password expiration helping or hu...
...,...,...,...
95,1891332177725456767,tut_ml,12 Best Courses to Learn Deep Learning!\nhttps...
96,1891331968819773592,Theo_Zou,RT @RemoteSens_MDPI: #mostcited\nEstimation of...
97,1891331897785008232,tut_ml,FREE Statistics Courses- https://t.co/04Hg1nfO...
98,1891331745250730163,kdagiit,"Data prep is like cooking, model training is l..."


In [63]:
df = df.dropna()
df

,tweet_id,username,text
0,1891299408085401715,Paul_hack13,You need help with your account recovery kindl...
1,1891299346622148795,Property_Report,Updates: Phuket luxury market lures foreign in...
2,1891299238291611988,notredamuz9,"@MrZackMorris ETF500's price may be volatile, ..."
3,1891297982756102281,Webcrack01,Is mandatory password expiration helping or hu...
4,1891297817269788738,Recoverytheate,Is mandatory password expiration helping or hu...
...,...,...,...
95,1891332177725456767,tut_ml,12 Best Courses to Learn Deep Learning!\nhttps...
96,1891331968819773592,Theo_Zou,RT @RemoteSens_MDPI: #mostcited\nEstimation of...
97,1891331897785008232,tut_ml,FREE Statistics Courses- https://t.co/04Hg1nfO...
98,1891331745250730163,kdagiit,"Data prep is like cooking, model training is l..."


In [64]:
#Clean the tweet text
df['cleaned_text'] = df['text'].apply(clean_tweet)
df

,tweet_id,username,text,cleaned_text
0,1891299408085401715,Paul_hack13,You need help with your account recovery kindl...,need help account recovery kindly send dm arti...
1,1891299346622148795,Property_Report,Updates: Phuket luxury market lures foreign in...,updates phuket luxury market lures foreign inv...
2,1891299238291611988,notredamuz9,"@MrZackMorris ETF500's price may be volatile, ...",etfs price may volatile one things sure public...
3,1891297982756102281,Webcrack01,Is mandatory password expiration helping or hu...,mandatory password expiration helping hurting ...
4,1891297817269788738,Recoverytheate,Is mandatory password expiration helping or hu...,mandatory password expiration helping hurting ...
...,...,...,...,...
95,1891332177725456767,tut_ml,12 Best Courses to Learn Deep Learning!\nhttps...,best courses learn deep learning machinelearni...
96,1891331968819773592,Theo_Zou,RT @RemoteSens_MDPI: #mostcited\nEstimation of...,rt mostcited estimation surface concentrations...
97,1891331897785008232,tut_ml,FREE Statistics Courses- https://t.co/04Hg1nfO...,free statistics courses machinelearning deeple...
98,1891331745250730163,kdagiit,"Data prep is like cooking, model training is l...",data prep like cooking model training like bak...


In [65]:
# emove empty tweets after cleaning
df = df[df['cleaned_text'].str.strip().str.len() > 0]
df

,tweet_id,username,text,cleaned_text
0,1891299408085401715,Paul_hack13,You need help with your account recovery kindl...,need help account recovery kindly send dm arti...
1,1891299346622148795,Property_Report,Updates: Phuket luxury market lures foreign in...,updates phuket luxury market lures foreign inv...
2,1891299238291611988,notredamuz9,"@MrZackMorris ETF500's price may be volatile, ...",etfs price may volatile one things sure public...
3,1891297982756102281,Webcrack01,Is mandatory password expiration helping or hu...,mandatory password expiration helping hurting ...
4,1891297817269788738,Recoverytheate,Is mandatory password expiration helping or hu...,mandatory password expiration helping hurting ...
...,...,...,...,...
95,1891332177725456767,tut_ml,12 Best Courses to Learn Deep Learning!\nhttps...,best courses learn deep learning machinelearni...
96,1891331968819773592,Theo_Zou,RT @RemoteSens_MDPI: #mostcited\nEstimation of...,rt mostcited estimation surface concentrations...
97,1891331897785008232,tut_ml,FREE Statistics Courses- https://t.co/04Hg1nfO...,free statistics courses machinelearning deeple...
98,1891331745250730163,kdagiit,"Data prep is like cooking, model training is l...",data prep like cooking model training like bak...


In [66]:
# Save the cleaned dataset
cleaned_file = "tweets_cleaned.csv"
df.to_csv(cleaned_file, index=False)

# Print summary statistics
print(f"Original tweet count: {len(df)}")
print(f"Tweets after cleaning: {len(df)}")
print(f"Cleaned data saved to {cleaned_file}")

Original tweet count: 100
Tweets after cleaning: 100
Cleaned data saved to tweets_cleaned.csv


# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

The most challenging section was the 1st question as colab had a lot of issues when running selenium. I had to refer to stackoverflow to resolve this issue. The very first cell is a reference  - https://stackoverflow.com/questions/51046454/how-can-we-use-selenium-webdriver-in-colab-research-google-com


# Write your response below
Fill out survey and provide your valuable feedback.

https://docs.google.com/forms/d/e/1FAIpQLSd_ObuA3iNoL7Az_C-2NOfHodfKCfDzHZtGRfIker6WyZqTtA/viewform?usp=dialog